In [37]:
import pyranges as pr
import pandas as pd
import numpy as np
import h5py 
import os

from cerberus.cerberus import *
from cerberus.main import *


## modify / collapse transcripts in TALON abundance file

In [148]:
# def map_transcripts(df, m_df, tname, tid):
#     """
#     df (pandas DataFrame): DataFrame with transcript ids
#     m_df (pandas DataFrame): Map DataFrame from cerberus reference
#     tname (str): Column name for transcript name in df
#     tid (str): Column name for transcript id in df
#     """
    
#     # fix transcript ids in abundance file
#     t_map = m_df[['original_transcript_id', 'original_transcript_name',
#                   'transcript_name', 'transcript_id']]
#     df = df.merge(t_map, left_on=tid, right_on='original_transcript_id')

#     df.drop(['original_transcript_id', 'original_transcript_name',
#              tid, tname],
#             axis=1, inplace=True)
#     df.rename({'transcript_name': tname,
#                'transcript_id': tid},
#               axis=1, inplace=True)
    
#     return df

In [114]:
def test_map_transcripts(print_dfs=True):
    # map file
    otid = ['1', '2', '3', '4']
    otname = ['1n', '2n', '3n', '4n']
    tid = ['1[1,1,1]', '2[1,1,1]', '3[1,1,1]', '4[1,1,1]']
    tname = ['1n[1,1,1]', '2n[1,1,1]', '3n[1,1,1]', '4n[1,1,1]']
    m_df = pd.DataFrame()
    m_df['original_transcript_id'] = otid
    m_df['original_transcript_name'] = otname
    m_df['transcript_id'] = tid
    m_df['transcript_name'] = tname

    # transcript file
    tid = ['2', '3', '4', '1']
    tname = ['2n', '3n', '4n', '1n']
    df = pd.DataFrame()
    df['annot_transcript_id'] = tid
    df['annot_transcript_name'] = tname
    df['tid_2'] = tid

    # ctrl 
    otid = ['2', '3', '4', '1']
    tname = ['2n[1,1,1]', '3n[1,1,1]', '4n[1,1,1]', '1n[1,1,1]'] 
    tid = ['2[1,1,1]', '3[1,1,1]', '4[1,1,1]', '1[1,1,1]']
    ctrl = pd.DataFrame()
    ctrl['tid_2'] = otid
    ctrl['annot_transcript_id'] = tid
    ctrl['annot_transcript_name'] = tname

    test = map_transcripts(df, m_df, 'annot_transcript_name', 'annot_transcript_id')

    if print_dfs:
        print('test')
        print(test)
        print(test.index)
        print(test.dtypes)
        print('ctrl')
        print(ctrl)
        print(ctrl.index)
        print(ctrl.dtypes)

    pd.testing.assert_frame_equal(ctrl, test, check_like=True)

    assert len(ctrl.index) == len(test.index)

In [115]:
test_map_transcripts(print_dfs=False)

In [121]:
# def agg_ab(df):
#     """
#     Aggregates expression over transcripts that now are the same based on 
#     assignment to cerberus reference ends
    
#     Parameters:
#         df (pandas DataFrame): TALON abundance file output from `map_transcripts`
    
#     Returns:
#         df (pandas DataFrame): Abundance file with counts and other metadata
#             per transcript aggregated for duplicated transcripts
#     """
#     gb_cols = ['gene_ID', 'annot_gene_id', 'annot_gene_name',
#                'gene_novelty', 'annot_transcript_name',
#                'annot_transcript_id']

#     # handle properties which won't always correspond across the transcripts
#     # these are all subject to change
#     nov_rank, rank_nov = get_nov_ranks()
#     df['nov_rank'] = df.transcript_novelty.map(nov_rank)
#     df.drop('transcript_novelty', axis=1, inplace=True)

#     ism_rank, rank_ism = get_ism_ranks()
#     df['ism_rank'] = df.ISM_subtype.map(ism_rank)
#     df.drop('ISM_subtype', axis=1, inplace=True)

#     df['transcript_ID'] = df.transcript_ID.astype(str)
#     agg_dict = {'transcript_ID': ','.join,
#                 'n_exons': 'mean',
#                 'length': 'mean',
#                 'nov_rank': 'min',
#                 'ism_rank': 'min'}
#     cols = gb_cols + list(agg_dict.keys())
#     for c in list(set(df.columns)-set(cols)):
#         agg_dict[c] = 'sum'

#     df = df.groupby(gb_cols).agg(agg_dict).reset_index()
#     df['transcript_novelty'] = df.nov_rank.map(rank_nov)
#     df.drop(['nov_rank'], axis=1, inplace=True)
#     df['ISM_subtype'] = df.ism_rank.map(rank_ism)
#     df.drop(['ism_rank'], axis=1, inplace=True)

#     return df

In [124]:
# def replace_ab_ids(ab, h5, agg):
#     """
#     Replace the transcript ids and transcript names in a TALON abundance file
#     with the new transcript ids that contain the triplet

#     Parameters:
#         ab (str): Path to TALON abundance file
#         h5 (str): Path to h5 annotation (output from assign)
#         agg (bool): Aggregate / collapse transcripts with the same triplets
#             and sum up their count values

#     Returns:
#         df (pandas DataFrame): TALON abundance file with updated
#             transcript ids / names
#     """
#     df = pd.read_csv(ab, sep='\t')
#     _, _, _, m_df = read_h5(h5)
    
#     df = map_transcripts(df, m_df, 'annot_transcript_name', 'annot_transcript_id')

#     # aggregate counts if requested
#     if agg:
#         df = agg_ab(df)

#     # reorder columns
#     c1 = get_non_dataset_cols()
#     c2 = get_dataset_cols(df)
#     df = df[c1+c2]

#     return df

In [144]:
def make_talon_ab(t,ta,n,i,e,l,ab):
    df = pd.DataFrame()
    cols = ['gene_ID', 'annot_gene_id', 'annot_gene_name',
                   'gene_novelty', 'annot_transcript_name',
                   'annot_transcript_id', 'transcript_ID']
    for c in cols:
        df[c] = t
    df['transcript_ID'] = ta
    df['transcript_novelty'] = n
    df['ISM_subtype'] = i
    df['n_exons'] = e
    df['length'] = l
    for i, a in enumerate(ab):
        c = 'ab{}'.format(i)
        df[c] = a
    
    return df
    
def test_agg_ab(print_dfs=True):
    # input
    t = ['1', '2', '2', '3', '3']
    ta = ['5', '6', '7', '8', '9']
    n = ['Known', 'Known', 'NIC', 'ISM', 'ISM']
    i = [None, None, None, 'Prefix', 'Both']
    e = [1,1,3,1,1]
    l = [1,1,5,1,1]
    ab = [[0,1,1,2,2],
          [3,0,1,1,2]]
    df = make_talon_ab(t,ta,n,i,e,l,ab)

    # ctrl
    t = ['1', '2', '3']
    ta = ['5', '6,7', '8,9']
    n = ['Known', 'Known', 'ISM']
    i = [None, None, 'Both']
    e = [1,2,1]
    l = [1,3,1]
    ab = [[0,2,4],
          [3,1,3]]
    ctrl = make_talon_ab(t,ta,n,i,e,l,ab)

    test = agg_ab(df)

    col_order = ctrl.columns.tolist()
    col_order.sort()
    test = test[col_order]
    ctrl = ctrl[col_order]
    test.sort_values(by='annot_transcript_id', inplace=True)
    ctrl.sort_values(by='annot_transcript_id', inplace=True)
    test.reset_index(inplace=True, drop=True)
    ctrl.reset_index(inplace=True, drop=True)
    ctrl.n_exons = ctrl.n_exons.astype(float)
    ctrl.length = ctrl.length.astype(float)


    if print_dfs:
        print('test')
        print(test)
        print(test.index)
        print(test.dtypes)
        print('ctrl')
        print(ctrl)
        print(ctrl.index)
        print(ctrl.dtypes)

    pd.testing.assert_frame_equal(ctrl, test, check_like=True)

    assert len(ctrl.index) == len(test.index)

In [147]:
test_agg_ab(print_dfs=False)

In [130]:
test.head()

,gene_ID,annot_gene_id,annot_gene_name,gene_novelty,annot_transcript_name,annot_transcript_id,transcript_ID,n_exons,length,ab1,ab0,transcript_novelty,ISM_subtype
0,1,1,1,1,1,1,5,1.0,1.0,3,0,Known,None
1,2,2,2,2,2,2,"6,7",2.0,3.0,1,2,Known,None
2,3,3,3,3,3,3,"8,9",1.0,1.0,3,4,ISM,Both


In [131]:
ctrl

,gene_ID,annot_gene_id,annot_gene_name,gene_novelty,annot_transcript_name,annot_transcript_id,transcript_ID,transcript_novelty,ISM_subtype,n_exons,length,ab0,ab1
0,1,1,1,1,1,1,5,Known,None,1,1,0,3
1,2,2,2,2,2,2,"6,7",Known,None,2,3,2,1
2,3,3,3,3,3,3,"8,9",ISM,Both,1,1,4,3


In [117]:
h5 = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/human_cerberus.h5'
ab = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/talon/human_talon_abundance_filtered.tsv'
agg = True

# replace_ab_ids(ab, h5, agg)

,gene_ID,transcript_ID,annot_gene_id,annot_transcript_id,annot_gene_name,annot_transcript_name,n_exons,length,gene_novelty,transcript_novelty,...,right_cardiac_atrium_4_1,imr90_1_1,mcf10a_1_1,h9_chondro_1_3,h9_de_1_1,gm12878_1_4,adrenal_gland_2_1,gm12878_3_1,h9_osteocyte_1_2,mucosa_of_descending_colon_2_1
0,2,1150534,ENSG00000227232.5,"ENSG00000227232[1,2,1]",WASH7P,"WASH7P[1,2,1]",9.0,2065.0,Known,NIC,...,0,0,0,0,0,0,1,0,0,2
1,2,207076,ENSG00000227232.5,"ENSG00000227232[1,3,1]",WASH7P,"WASH7P[1,3,1]",10.0,1828.0,Known,NIC,...,0,3,0,0,0,0,0,0,0,0
2,2,207057,ENSG00000227232.5,"ENSG00000227232[1,4,1]",WASH7P,"WASH7P[1,4,1]",10.0,2164.0,Known,NIC,...,2,1,0,0,0,0,0,0,0,3
3,2,207067,ENSG00000227232.5,"ENSG00000227232[1,5,1]",WASH7P,"WASH7P[1,5,1]",11.0,1922.0,Known,NIC,...,2,0,0,1,1,0,1,0,0,2
4,3,4,ENSG00000278267.1,"ENSG00000278267[1,1,1]",MIR6859-1,"MIR6859-1[1,1,1]",1.0,68.0,Known,Known,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132933,521227,5366801,ENCODEHG000521227,"ENCODEHG000521227[1,1,1]",ENCODEHG000521227,"ENCODEHG000521227[1,1,1]",2.0,329.0,Antisense,NNC,...,0,0,0,0,0,0,0,0,0,0
132934,570465,5023728,ENCODEHG000570465,"ENCODEHG000570465[1,1,1]",ENCODEHG000570465,"ENCODEHG000570465[1,1,1]",2.0,2830.0,Intergenic,NNC,...,0,0,0,0,0,0,0,0,0,0
132935,573849,5483046,ENCODEHG000573849,"ENCODEHG000573849[1,1,1]",ENCODEHG000573849,"ENCODEHG000573849[1,1,1]",5.0,938.0,Intergenic,NNC,...,0,0,0,0,0,0,0,0,0,0
132936,576557,5166770,ENCODEHG000576557,"ENCODEHG000576557[1,1,1]",ENCODEHG000576557,"ENCODEHG000576557[1,1,1]",2.0,658.0,Intergenic,NNC,...,0,0,0,0,0,0,0,0,0,0


In [87]:
df = pd.read_csv(ab, sep='\t')
_, _, _, m_df = read_h5(h5)

In [91]:
# fix transcript ids in abundance file
ab_map = m_df[['original_transcript_id', 'original_transcript_name', 'transcript_name', 'transcript_id']]
df = df.merge(ab_map, left_on='tid', right_on='original_transcript_id')

df.drop(['original_transcript_id', 'original_transcript_name',
         'tid', tname],
        axis=1, inplace=True)
df.rename({'transcript_name': tname,
           'transcript_id': 'tid'},
          axis=1, inplace=True)

In [92]:
df.head()

,gene_ID,transcript_ID,annot_gene_id,annot_gene_name,n_exons,length,gene_novelty,transcript_novelty,ISM_subtype,gm12878_1_1,...,brodmann_area_46_7_1,brodmann_area_46_5_1,brodmann_area_46_6_1,brodmann_area_46_8_1,brodmann_area_46_4_1,brodmann_area_46_2_1,brodmann_area_46_9_1,brodmann_area_46_3_1,annot_transcript_name,annot_transcript_id
0,3,4,ENSG00000278267.1,MIR6859-1,1,68,Known,Known,None,0,...,0,0,0,0,0,0,0,0,"MIR6859-1 [1,1,1]","ENSG00000278267 [1,1,1]"
1,4,6,ENSG00000243485.5,MIR1302-2HG,2,535,Known,Known,None,0,...,0,0,0,0,0,0,0,0,"MIR1302-2HG [3,2,1]","ENSG00000243485 [3,2,1]"
2,6,8,ENSG00000237613.2,FAM138A,3,1187,Known,Known,None,0,...,0,0,0,0,0,0,0,0,"FAM138A [1,1,1]","ENSG00000237613 [1,1,1]"
3,6,9,ENSG00000237613.2,FAM138A,2,590,Known,Known,None,0,...,0,0,0,0,0,0,0,0,"FAM138A [1,2,2]","ENSG00000237613 [1,2,2]"
4,10,19,ENSG00000238009.6,AL627309.1,2,336,Known,Known,None,0,...,0,0,0,0,0,0,0,0,"AL627309.1 [4,5,5]","ENSG00000238009 [4,5,5]"
